# Training a simple CNN model in Tensorflow for Tornado Detection

This notebook steps through how to train a simple CNN model using a subset of TorNet.

This will not produce a model with any skill, but simply provides a working end-to-end example of how to set up a data loader, build, and fit a model


In [1]:
import sys
# Uncomment if tornet isn't installed in your environment or in your path already
#sys.path.append('../')  

import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tornet.data.tf.loader import create_tf_dataset 
from tornet.data.constants import ALL_VARIABLES
import tornet.data.preprocess as pp
from tornet.data import preprocess as tfpp
import keras
from tornet.data.constants import CHANNEL_MIN_MAX
import tornet.metrics.keras.metrics as km

In [2]:
# Create basic dataloader
# This option loads directly from netcdf files, and will be slow and IO bound
# To speed up training, either
#     build as a tensorflow_dataset , (see tornet/data/tfds/tornet/README.md)
#     cache dataset first , or
#     use tf.data.Dataset.load on a pre-saved dataset

# Location of tornet
data_root = 'C:/Users/mjhig/tornet_2013'

# Get training data from 2018
data_type='train'
years = [2013,]

catalog_path = os.path.join(data_root,'catalog.csv')
if not os.path.exists(catalog_path):
    raise RuntimeError('Unable to find catalog.csv at '+data_root)
        
catalog = pd.read_csv(catalog_path,parse_dates=['start_time','end_time'])
catalog = catalog[catalog['type']==data_type]
catalog = catalog[catalog.start_time.dt.year.isin(years)]
catalog = catalog.sample(frac=1,random_state=1234)
file_list = [os.path.join(data_root,f) for f in catalog.filename]

ds = create_tf_dataset(file_list,variables=ALL_VARIABLES,n_frames=1) 

# (Optional) Save data for faster reloads (makes copy of data!)
# ds.save('tornet_sample.tfdataset') 


In [3]:
# If saved with ds.save(...), just load that model
#ds = tf.data.Dataset.load('tornet_sample.tfdataset')

In [4]:
# If data was registered in tensorflow_dataset, use that
# env variable TFDS_DATA_DIR should point to location of this resaved dataset
#import tensorflow_datasets as tfds
#import tornet.data.tfds.tornet.tornet_dataset_builder # registers 'tornet'

#data_type='train'
#years = [2013,]
#ds = tfds.load('tornet',split='+'.join(['%s-%d' % (data_type,y) for y in years]))

In [5]:
# Preprocess

# add 'coordinates' variable used by CoordConv layers
ds = ds.map(lambda d: pp.add_coordinates(d,include_az=False,backend=tf))
     
# Take only last time frame
ds = ds.map(pp.remove_time_dim)

# Split sample into inputs,label
ds = ds.map(tfpp.split_x_y)

# (Optional) add sample weights
# weights={'wN':1.0,'w0':1.0,'w1':1.0,'w2':2.0,'wW':0.5}
# ds = ds.map(lambda x,y:  tfpp.compute_sample_weight(x,y,**weights) )

ds = ds.prefetch(tf.data.AUTOTUNE)
        
ds = ds.batch(32)



In [20]:
# Create a simple CNN model
# This normalizes data, concatenates along channel, and applies a Conv2D


input_vars = ALL_VARIABLES # which variables to use

# TF convention is B,L,W,H
inputs = {v:keras.Input(shape=(120,240,2),name=v) for v in input_vars}

# Normalize inputs
norm_layers = []
for v in input_vars:
    min_max = np.array(CHANNEL_MIN_MAX[v]) # [2,]

    # choose mean,var to get approximate [-1,1] scaling
    var=((min_max[1]-min_max[0])/2)**2 # scalar
    var=np.array(2*[var,])    # [n_sweeps,]
    offset=(min_max[0]+min_max[1])/2    # scalar
    offset=np.array(2*[offset,]) # [n_sweeps,]
    
    norm_layers.append(
        keras.layers.Normalization(mean=offset, variance=var,
                                   name='Normalized_%s' % v)
    )

# Concatenate normed inputs along channel dimension
x=keras.layers.Concatenate(axis=-1,name='Concatenate1')(
        [l(inputs[v]) for l,v in zip(norm_layers,input_vars)]
        )

# Replace background (nan) with -3
x=keras.layers.Lambda(lambda x: tf.where(tf.math.is_nan(x),-3.0,x),name='ReplaceNan')(x)

# Processing
x = keras.layers.Conv2D(32,3,padding='same',activation='relu')(x)
# add more..
x = keras.layers.Conv2D(1,1,padding='same',activation='relu',name='TornadoLikelihood')(x)
y = keras.layers.GlobalMaxPool2D(name='GlobalMaxPool')(x)

model = keras.Model(inputs=inputs,outputs=y,name='TornadoDetector')

model.summary()

Model: "TornadoDetector"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ DBZ (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ VEL (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ KDP (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ RHOHV (InputLayer)  │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ZDR (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ WIDTH (InputLayer)  │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_DBZ      │ (None, 120, 240,  │          0 │ DBZ[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_VEL      │ (None, 120, 240,  │          0 │ VEL[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_KDP      │ (None, 120, 240,  │          0 │ KDP[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_RHOHV    │ (None, 120, 240,  │          0 │ RHOHV[0][0]       │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_ZDR      │ (None, 120, 240,  │          0 │ ZDR[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalized_WIDTH    │ (None, 120, 240,  │          0 │ WIDTH[0][0]       │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Concatenate1        │ (None, 120, 240,  │          0 │ Normalized_DBZ[0… │
│ (Concatenate)       │ 12)               │            │ Normalized_VEL[0… │
│                     │                   │            │ Normalized_KDP[0… │
│                     │                   │            │ Normalized_RHOHV… │
│                     │                   │            │ Normalized_ZDR[0… │
│                     │                   │            │ Normalized_WIDTH… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ReplaceNan (Lambda) │ (None, 120, 240,  │          0 │ Concatenate1[0][… │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 120, 240,  │      3,488 │ ReplaceNan[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 3,521 (13.75 KB)

 Trainable params: 3,521 (13.75 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Compile
#This creates an instance of the Adam optimizer, 
#a popular optimization algorithm in machine learning that adapts the learning rate for each parameter. 
#The learning rate (1e-3 or 0.001) controls how large the updates to the model's weights will be during training.
opt  = keras.optimizers.Adam(learning_rate=1e-3)
#This sets the loss function to BinaryCrossentropy, which is used for binary classification problems. 
# The from_logits=True parameter specifies that the model's output is raw scores (logits) and not probabilities, 
# so the loss function will apply a sigmoid activation internally.
loss=keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer=opt)

In [28]:
# Train
# steps_per_epoch=10 for demo purposes
model.fit(ds,epochs=3,steps_per_epoch=10)

Epoch 1/3
10/10 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - loss: 0.8554
Epoch 2/3
10/10 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - loss: 0.7118
Epoch 3/3
10/10 ━━━━━━━━━━━━━━━━━━━━ 38s 4s/step - loss: 0.6981


In [83]:
# Build a test set
# Basic loader
data_type='test'
years = [2013]

catalog_path = os.path.join(data_root,'catalog.csv')
if not os.path.exists(catalog_path):
    raise RuntimeError('Unable to find catalog.csv at '+data_root)
        
catalog = pd.read_csv(catalog_path,parse_dates=['start_time','end_time'])
catalog = catalog[catalog['type']==data_type]
catalog = catalog[catalog.start_time.dt.year.isin(years)]
catalog = catalog.sample(frac=1,random_state=1234)
file_list = [os.path.join(data_root,f) for f in catalog.filename]

ds_test = create_tf_dataset(file_list,variables=ALL_VARIABLES,n_frames=1) 


In [82]:
# preprocess
ds_test = ds_test.map(lambda d: pp.add_coordinates(d,include_az=False,backend=tf))
ds_test = ds_test.map(pp.remove_time_dim)
ds_test = ds_test.map(tfpp.split_x_y)
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(32)


TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.<lambda>() takes 1 positional argument but 2 were given


In [79]:
# Evaluate
import tornet.metrics.keras.metrics as km
metrics = [keras.metrics.AUC(from_logits=True,name='AUC'),
           keras.metrics.BinaryAccuracy(name='BinaryAccuracy'),
           km.Precision(name='Precision'),
           km.Recall(name='Recall'),
           km.F1Score(name='F1Score')]
model.compile(loss=loss,metrics=metrics)

# steps=10 for demo purposes
model.evaluate(ds_test,steps=10)


c:\Users\mjhig\anaconda3\envs\tornet-torch\lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: {'DBZ': 'DBZ', 'VEL': 'VEL', 'KDP': 'KDP', 'RHOHV': 'RHOHV', 'ZDR': 'ZDR', 'WIDTH': 'WIDTH', 'range_folded_mask': 'range_folded_mask', 'coordinates': 'coordinates'}. Received: the structure of inputs={'DBZ': '*', 'VEL': '*', 'KDP': '*', 'RHOHV': '*', 'ZDR': '*', 'WIDTH': '*', 'range_folded_mask': '*', 'label': '*', 'category': '*', 'event_id': '*', 'ef_number': '*', 'az_lower': '*', 'az_upper': '*', 'rng_lower': '*', 'rng_upper': '*', 'time': '*', 'tornado_start_time': '*', 'tornado_end_time': '*', 'coordinates': '*'}
  warnings.warn(


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("IteratorGetNext:6", shape=(None,), dtype=float32). Expected shape (None, None, None, 2), but input has incompatible shape (None,)[0m

Arguments received by Functional.call():
  • inputs={'DBZ': 'tf.Tensor(shape=(None, 120, 240, 2), dtype=float32)', 'VEL': 'tf.Tensor(shape=(None, 120, 240, 2), dtype=float32)', 'KDP': 'tf.Tensor(shape=(None, 120, 240, 2), dtype=float32)', 'RHOHV': 'tf.Tensor(shape=(None, 120, 240, 2), dtype=float32)', 'ZDR': 'tf.Tensor(shape=(None, 120, 240, 2), dtype=float32)', 'WIDTH': 'tf.Tensor(shape=(None, 120, 240, 2), dtype=float32)', 'range_folded_mask': 'tf.Tensor(shape=(None, 120, 240, 2), dtype=float32)', 'label': 'tf.Tensor(shape=(None,), dtype=uint8)', 'category': 'tf.Tensor(shape=(None,), dtype=int64)', 'event_id': 'tf.Tensor(shape=(None,), dtype=int64)', 'ef_number': 'tf.Tensor(shape=(None,), dtype=int64)', 'az_lower': 'tf.Tensor(shape=(None,), dty

In [80]:
from huggingface_hub import hf_hub_download
trained_model = hf_hub_download(repo_id="tornet-ml/tornado_detector_baseline_v1", 
                                filename="tornado_detector_baseline.keras")
model = keras.saving.load_model(trained_model,compile=False)

In [81]:
# Evaluate
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
import tornet.metrics.keras.metrics as km
metrics = [keras.metrics.AUC(from_logits=True,name='AUC'),
           keras.metrics.BinaryAccuracy(name='BinaryAccuracy'),
           km.Precision(name='Precision'),
           km.Recall(name='Recall'),
           km.F1Score(name='F1Score')]
model.compile(metrics=metrics,loss=loss)

# steps=10 for demo purposes
#model.trainable_variables
model.evaluate(ds_test,steps=10)

ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("IteratorGetNext:6", shape=(None,), dtype=float32). Expected shape (None, None, None, 2), but input has incompatible shape (None,)[0m

Arguments received by Functional.call():
  • inputs={'DBZ': 'tf.Tensor(shape=(None, 120, 240, 2), dtype=float32)', 'VEL': 'tf.Tensor(shape=(None, 120, 240, 2), dtype=float32)', 'KDP': 'tf.Tensor(shape=(None, 120, 240, 2), dtype=float32)', 'RHOHV': 'tf.Tensor(shape=(None, 120, 240, 2), dtype=float32)', 'ZDR': 'tf.Tensor(shape=(None, 120, 240, 2), dtype=float32)', 'WIDTH': 'tf.Tensor(shape=(None, 120, 240, 2), dtype=float32)', 'range_folded_mask': 'tf.Tensor(shape=(None, 120, 240, 2), dtype=float32)', 'label': 'tf.Tensor(shape=(None,), dtype=uint8)', 'category': 'tf.Tensor(shape=(None,), dtype=int64)', 'event_id': 'tf.Tensor(shape=(None,), dtype=int64)', 'ef_number': 'tf.Tensor(shape=(None,), dtype=int64)', 'az_lower': 'tf.Tensor(shape=(None,), dty